In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
data_path = "data/"

# Model building

Build models with:
- BoW or tf-idf features
- Raw review length
- Lix number [see here](https://en.wikipedia.org/wiki/Lix_(readability_test))
- Sentiment score (3 scores)

In [3]:
df = pd.read_csv("data/NLP_data.csv")
df['reviewTextClean'] = df['reviewTextClean'].str.lower()
df.dropna(subset=["reviewTextClean"], inplace=True)

In [5]:
df.head(2)

,rating,reviewText,reviewTextClean,posReviewPercent,negReviewPercent,midReviewPercent,price,LIX,NumberOfWords
0,3.0,This is a very cute hotel with good amenities ...,cute hotel good amenity nice location great cr...,0.141,0.109,0.751,3.0,28.500000,50
1,4.0,Love this place. The Great/Good: Massage an...,love place massage facial technician best loun...,0.234,0.051,0.716,3.0,82.047619,63


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
bow_counts = count_vect.fit_transform(df['reviewTextClean'].values)
bow_counts.shape

(96001, 33795)

In [105]:
print("Length of vocabulary:", len(count_vect.vocabulary_))
print("Length of corpus:", bow_counts.shape[0])

Length of vocabulary: 33795
Length of corpus: 96001


Create tf-idf representation of `reviewTextClean`

In [106]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
bow_tfidf = tfidf.fit_transform(bow_counts)

In [107]:
from sklearn.model_selection import train_test_split
y = df.price.values-1

# Split into development and test split
X_bow_dev, X_bow_test, X_tfidf_dev, X_tfidf_test, y_dev, y_test = train_test_split(bow_counts, bow_tfidf, y, test_size=0.2, stratify=y)

# Split development into training and validation
X_bow_train, X_bow_val, X_tfidf_train, X_tfidf_val, y_train, y_val = train_test_split(X_bow_dev, X_tfidf_dev, y_dev, test_size=0.1, stratify=y_dev)

In [108]:
# Function that returns performance of classifier 
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
def classification_metrics(header_text, clf, X_train, X_test, y_train, y_test):
    print("="*35)
    print(" "*int((35-len(header_text))/2), header_text)
    print("="*35)
    y_pred = clf.predict(X_train)
    print("Train Accuracy:", accuracy_score(y_train, y_pred))
    y_pred = clf.predict(X_test)
    print("Val Accuracy:", accuracy_score(y_test, y_pred))
    print("F1 Score:", f1_score(y_test, y_pred, average="weighted"))
    print(confusion_matrix(y_test, y_pred))

In [109]:
# Dummy classifier predicting most frequent class
from sklearn.dummy import DummyClassifier
dummy = DummyClassifier(strategy="most_frequent")
dummy.fit(X_bow_train, y_train)
y_pred = dummy.predict(X_bow_val)
classification_metrics("Dummy classifier", dummy, X_bow_train, X_bow_val, y_train, y_val)

          Dummy classifier
Train Accuracy: 0.5582465277777777
Val Accuracy: 0.558203125
F1 Score: 0.3999359566934069
[[   0 1382    0]
 [   0 4287    0]
 [   0 2011    0]]


In [110]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver="saga")
lr.fit(X_tfidf_train, y_train)
classification_metrics("LogisticRegression tf-idf", lr, X_tfidf_train, X_tfidf_val, y_train, y_val)

      LogisticRegression tf-idf
Train Accuracy: 0.7223379629629629
Val Accuracy: 0.6497395833333334
F1 Score: 0.630115823743557
[[ 491  835   56]
 [ 218 3605  464]
 [  35 1082  894]]


In [1]:
import xgboost
xgb = xgboost.XGBClassifier(eval_metric="mlogloss", use_label_encoder=False)
xgb.fit(X_tfidf_train, y_train)
classification_metrics("XGBoost tf-idf", xgb, X_tfidf_train, X_tfidf_val, y_train, y_val)

NameError: name 'X_tfidf_train' is not defined